In [1]:
import pickle as pkl
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')

# Model

In [2]:
with open('./pkl/train.pkl', 'rb') as f:
    train = pkl.load(f)

In [3]:
train.batter.values

array([  0,   1,   2, ..., 449, 377, 485])

In [4]:
batter_x = torch.LongTensor((train.batter.values)).view(-1, 1).cuda()
pitcher_x = torch.LongTensor(train.pitcher.values).view(-1, 1).cuda()
y = torch.LongTensor(pd.get_dummies(train.outcome).values).cuda()

In [5]:
batter_x.shape, pitcher_x.shape, y.shape

(torch.Size([1025441, 1]), torch.Size([1025441, 1]), torch.Size([1025441, 43]))

In [6]:
NUM_BATTERS = len(train.batter.unique())
NUM_PITCHERS = len(train.pitcher.unique())
NUM_OUTCOMES = len(train.outcome.unique())
VEC_SIZE = 9

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.hitter_layer = nn.Sequential(
            nn.Embedding(NUM_BATTERS, VEC_SIZE),
        )
        
        self.pitcher_layer = nn.Sequential(
            nn.Embedding(NUM_PITCHERS, VEC_SIZE),
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(VEC_SIZE * 2, NUM_OUTCOMES),
        )
        
    def forward(self, batter, pitcher):
        x1 = self.hitter_layer(batter)
        x1 = F.sigmoid(x1)
        x1 = x1.view(-1, VEC_SIZE)
        x2 = self.pitcher_layer(pitcher)
        x2 = F.sigmoid(x2)
        x2 = x2.view(-1, VEC_SIZE)
        x = torch.cat((x1, x2), dim=1)
        x = self.fc_layer(x)
        return x

In [7]:
model = Model().cuda()

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-6)

In [9]:
NUM_EPOCHS = 1024

In [10]:
cost_ls = []
acc_ls = []

In [11]:
for epoch in range(NUM_EPOCHS):
    pred = model(batter_x, pitcher_x)
    topk, idx = y.topk(1)
    predicted_classes = pred.max(1)[1]
    correct_count = (predicted_classes == idx.view(-1)).sum().item()
    acc_ls.append(correct_count / len(y) * 100)
    cost = loss(pred, idx.view(-1))
    cost_ls.append(cost.item())

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print('Epoch [%d/%d], Loss: %.4f, ACC: %.2f' 
             %(epoch+1, NUM_EPOCHS, cost.item(), correct_count / len(y) * 100))

Epoch [100/1024], Loss: 2.7681, ACC: 15.70
Epoch [200/1024], Loss: 2.7360, ACC: 16.24
Epoch [300/1024], Loss: 2.7216, ACC: 16.61
Epoch [400/1024], Loss: 2.7156, ACC: 16.71
Epoch [500/1024], Loss: 2.7123, ACC: 16.75
Epoch [600/1024], Loss: 2.7108, ACC: 16.77
Epoch [700/1024], Loss: 2.7100, ACC: 16.77
Epoch [800/1024], Loss: 2.7094, ACC: 16.77
Epoch [900/1024], Loss: 2.7090, ACC: 16.77
Epoch [1000/1024], Loss: 2.7087, ACC: 16.77


In [12]:
# with open('./model/weight.pkl', 'wb')as f:
#     pkl.dump(model.state_dict(), f)